In [234]:
import pandas as pd

In [235]:
import random

filename = "D:/Program/dataset/Taxi Trip Time Prediction/train.csv"

#number of records in file (excludes header)
df_rows = sum(1 for line in open(filename)) - 1 

# Sample size - in this case ~10%
size = int(df_rows / 1000)

#the 0-indexed header will not be included in the skip list
skip_idx = sorted(random.sample(range(1, df_rows), df_rows - size))

df = pd.read_csv(filename,skiprows=skip_idx)
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372651639620000517,A,2002.0,NaN,20000517,1372651639,A,False,"[[-8.583048,41.164632],[-8.582679,41.164173],[..."
1,1372687109620000616,C,NaN,NaN,20000616,1372687109,A,False,"[[-8.660061,41.148594],[-8.658504,41.148198],[..."
2,1372706176620000345,C,NaN,NaN,20000345,1372706176,A,False,"[[-8.632134,41.179131],[-8.630694,41.179266],[..."
3,1372768178620000251,B,NaN,7.0,20000251,1372768178,A,False,"[[-8.639829,41.159808],[-8.640405,41.15988],[-..."
4,1372848918620000148,A,2002.0,NaN,20000148,1372848918,A,False,"[[-8.604882,41.146659],[-8.60499,41.146668],[-..."


In [236]:
df.shape

(1710, 9)

In [237]:
df = df.drop(["CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TRIP_ID","DAY_TYPE","MISSING_DATA"],axis=1)
df.head()

,TAXI_ID,TIMESTAMP,POLYLINE
0,20000517,1372651639,"[[-8.583048,41.164632],[-8.582679,41.164173],[..."
1,20000616,1372687109,"[[-8.660061,41.148594],[-8.658504,41.148198],[..."
2,20000345,1372706176,"[[-8.632134,41.179131],[-8.630694,41.179266],[..."
3,20000251,1372768178,"[[-8.639829,41.159808],[-8.640405,41.15988],[-..."
4,20000148,1372848918,"[[-8.604882,41.146659],[-8.60499,41.146668],[-..."


In [238]:
import numpy as np
import ast

def caltime(row):
    
    polylen = len(ast.literal_eval(row["POLYLINE"]))
    timecost = (polylen-1)*15
    return timecost

def snapshot(row):
    snap = len(ast.literal_eval(row["POLYLINE"]))
    return snap

def initial_long(row):
    loc = ast.literal_eval(row["POLYLINE"])[0]
    longitude = loc[0]
    return longitude

def initial_lat(row):
    loc = ast.literal_eval(row["POLYLINE"])[0]
    latitude = loc[1]
    return latitude


In [239]:
df['TAXI_ID'] -= np.min(df['TAXI_ID']) 
df["timecost"] = df.apply(caltime,axis=1)
df["snapshots"] = df.apply(snapshot,axis=1)

df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots
0,516,1372651639,"[[-8.583048,41.164632],[-8.582679,41.164173],[...",405,28
1,615,1372687109,"[[-8.660061,41.148594],[-8.658504,41.148198],[...",435,30
2,344,1372706176,"[[-8.632134,41.179131],[-8.630694,41.179266],[...",645,44
3,250,1372768178,"[[-8.639829,41.159808],[-8.640405,41.15988],[-...",255,18
4,147,1372848918,"[[-8.604882,41.146659],[-8.60499,41.146668],[-...",405,28


In [240]:
empty_poly = df.loc[df['POLYLINE'] == '[]'].index.tolist()
df = df.drop(empty_poly)

snap_short = df.loc[df['snapshots']<4].index.tolist()
df = df.drop(snap_short)
df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots
0,516,1372651639,"[[-8.583048,41.164632],[-8.582679,41.164173],[...",405,28
1,615,1372687109,"[[-8.660061,41.148594],[-8.658504,41.148198],[...",435,30
2,344,1372706176,"[[-8.632134,41.179131],[-8.630694,41.179266],[...",645,44
3,250,1372768178,"[[-8.639829,41.159808],[-8.640405,41.15988],[-...",255,18
4,147,1372848918,"[[-8.604882,41.146659],[-8.60499,41.146668],[-...",405,28


In [241]:
from math import radians, cos, sin, asin, sqrt  
  
def haversine(row): 
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """   
    lon1 = row['initial_lon']
    lat1 = row['initial_lat']
    lon2 = ast.literal_eval(row["POLYLINE"])[-1][0]
    lat2 = ast.literal_eval(row["POLYLINE"])[-1][1]
    
    # change Decimal to radian   
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))
    
    # 地球平均半徑，單位為公里
    r = 6371
    diff = c * r * 1000  
    
    return diff/1000 

In [242]:
import time 

def get_date(row):
    tt = time.gmtime(row["TIMESTAMP"])
    time_data = [tt.tm_mon,tt.tm_mday,tt.tm_hour]
    
    return pd.Series(np.array(time_data, dtype=float))

In [243]:
df['initial_lon'] = df.apply(initial_long,axis=1)
df['initial_lat'] = df.apply(initial_lat,axis=1)
df['diff'] = df.apply(haversine,axis=1)

ds = df.apply(get_date,axis=1)
ds.columns = ["mon","day","hour"]
df = df.join(ds)

df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots,initial_lon,initial_lat,diff,mon,day,hour
0,516,1372651639,"[[-8.583048,41.164632],[-8.582679,41.164173],[...",405,28,-8.583048,41.164632,1.767325,7.0,1.0,4.0
1,615,1372687109,"[[-8.660061,41.148594],[-8.658504,41.148198],[...",435,30,-8.660061,41.148594,0.808887,7.0,1.0,13.0
2,344,1372706176,"[[-8.632134,41.179131],[-8.630694,41.179266],[...",645,44,-8.632134,41.179131,3.212306,7.0,1.0,19.0
3,250,1372768178,"[[-8.639829,41.159808],[-8.640405,41.15988],[-...",255,18,-8.639829,41.159808,1.279579,7.0,2.0,12.0
4,147,1372848918,"[[-8.604882,41.146659],[-8.60499,41.146668],[-...",405,28,-8.604882,41.146659,0.764345,7.0,3.0,10.0


In [244]:
cols = df.columns.tolist()
y_label = cols[3]
del cols[3]
cols.append(y_label)
df = df[cols]

df = df.drop(["TIMESTAMP"],axis=1)
df = df.drop(["POLYLINE"],axis=1)
df = df.drop(["TAXI_ID"],axis=1)
df.head()

,snapshots,initial_lon,initial_lat,diff,mon,day,hour,timecost
0,28,-8.583048,41.164632,1.767325,7.0,1.0,4.0,405
1,30,-8.660061,41.148594,0.808887,7.0,1.0,13.0,435
2,44,-8.632134,41.179131,3.212306,7.0,1.0,19.0,645
3,18,-8.639829,41.159808,1.279579,7.0,2.0,12.0,255
4,28,-8.604882,41.146659,0.764345,7.0,3.0,10.0,405


In [245]:
X_train,Y_train = df[df.columns[:-1]],df[df.columns[-1]]
X_train = X_train.drop(["mon","day"],axis=1)

X_train.head()

,snapshots,initial_lon,initial_lat,diff,hour
0,28,-8.583048,41.164632,1.767325,4.0
1,30,-8.660061,41.148594,0.808887,13.0
2,44,-8.632134,41.179131,3.212306,19.0
3,18,-8.639829,41.159808,1.279579,12.0
4,28,-8.604882,41.146659,0.764345,10.0


In [246]:
from keras.models import Sequential  
from keras.layers import Dense

In [251]:
model = Sequential() 
model.add(Dense(units=50,input_shape=(5,),activation="relu",name="hidden_1"))
model.add(Dense(units=50,activation="relu",name="hidden_2"))
model.add(Dense(units=50,activation="relu",name="hidden_3"))
model.add(Dense(units=50,activation="relu",name="hidden_4"))
model.add(Dense(units=1,name="output"))
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_1 (Dense)             (None, 50)                300       
_________________________________________________________________
hidden_2 (Dense)             (None, 50)                2550      
_________________________________________________________________
hidden_3 (Dense)             (None, 50)                2550      
_________________________________________________________________
hidden_4 (Dense)             (None, 50)                2550      
_________________________________________________________________
output (Dense)               (None, 1)                 51        
Total params: 8,001
Trainable params: 8,001
Non-trainable params: 0
_________________________________________________________________


In [252]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
train_history = model.fit(X_train.values, Y_train.values, validation_split=0.2, epochs=3000, batch_size=20, verbose=1) 

Train on 1338 samples, validate on 335 samples
Epoch 1/3000
1338/1338 [==============================] - 1s - loss: 640063.4004 - acc: 7.4738e-04 - val_loss: 80826.2546 - val_acc: 0.0030
Epoch 2/3000
1338/1338 [==============================] - 0s - loss: 41627.2845 - acc: 0.0030 - val_loss: 5526.7608 - val_acc: 0.0090
Epoch 3/3000
1338/1338 [==============================] - 0s - loss: 1992.3928 - acc: 0.0127 - val_loss: 928.7029 - val_acc: 0.0000e+00
Epoch 4/3000
1338/1338 [==============================] - 0s - loss: 562.4129 - acc: 0.0157 - val_loss: 437.4259 - val_acc: 0.0209
Epoch 5/3000
1338/1338 [==============================] - 0s - loss: 279.6774 - acc: 0.0269 - val_loss: 220.0103 - val_acc: 0.0179
Epoch 6/3000
1338/1338 [==============================] - 0s - loss: 165.0713 - acc: 0.0411 - val_loss: 133.3116 - val_acc: 0.0119
Epoch 7/3000
1338/1338 [==============================] - 0s - loss: 102.0088 - acc: 0.0478 - val_loss: 72.5132 - val_acc: 0.0299
Epoch 8/3000
1338/13

1338/1338 [==============================] - 0s - loss: 8.0932 - acc: 0.2040 - val_loss: 7.6938 - val_acc: 0.0507
Epoch 65/3000
1338/1338 [==============================] - 0s - loss: 12.6734 - acc: 0.1487 - val_loss: 1.5706 - val_acc: 0.2328
Epoch 66/3000
1338/1338 [==============================] - 0s - loss: 2.9043 - acc: 0.2593 - val_loss: 3.6960 - val_acc: 0.1910
Epoch 67/3000
1338/1338 [==============================] - 0s - loss: 3.5704 - acc: 0.2616 - val_loss: 1.3119 - val_acc: 0.2418
Epoch 68/3000
1338/1338 [==============================] - 0s - loss: 3.6769 - acc: 0.2862 - val_loss: 22.6910 - val_acc: 0.0030
Epoch 69/3000
1338/1338 [==============================] - 0s - loss: 7.9548 - acc: 0.1779 - val_loss: 3.7945 - val_acc: 0.1075
Epoch 70/3000
1338/1338 [==============================] - 0s - loss: 0.8227 - acc: 0.5202 - val_loss: 0.5698 - val_acc: 0.6597
Epoch 71/3000
1338/1338 [==============================] - 0s - loss: 1.5584 - acc: 0.4679 - val_loss: 4.6030 - val_

1338/1338 [==============================] - 0s - loss: 2.8141 - acc: 0.2750 - val_loss: 11.2155 - val_acc: 0.0537
Epoch 128/3000
1338/1338 [==============================] - 0s - loss: 4.0989 - acc: 0.2384 - val_loss: 6.0120 - val_acc: 0.2179
Epoch 129/3000
1338/1338 [==============================] - 0s - loss: 5.9801 - acc: 0.3117 - val_loss: 1.0929 - val_acc: 0.4060
Epoch 130/3000
1338/1338 [==============================] - 0s - loss: 7.9619 - acc: 0.2795 - val_loss: 5.8514 - val_acc: 0.0806
Epoch 131/3000
1338/1338 [==============================] - 0s - loss: 16.0317 - acc: 0.1345 - val_loss: 8.0163 - val_acc: 0.1881
Epoch 132/3000
1338/1338 [==============================] - 0s - loss: 3.5812 - acc: 0.2818 - val_loss: 1.5344 - val_acc: 0.3761
Epoch 133/3000
1338/1338 [==============================] - 0s - loss: 7.0009 - acc: 0.2354 - val_loss: 35.8487 - val_acc: 0.0060
Epoch 134/3000
1338/1338 [==============================] - 0s - loss: 32.4170 - acc: 0.1398 - val_loss: 1.75

1338/1338 [==============================] - 0s - loss: 1.5150 - acc: 0.4320 - val_loss: 0.6090 - val_acc: 0.5910
Epoch 191/3000
1338/1338 [==============================] - 0s - loss: 9.8670 - acc: 0.2496 - val_loss: 2.7124 - val_acc: 0.1463
Epoch 192/3000
1338/1338 [==============================] - 0s - loss: 175.6332 - acc: 0.0852 - val_loss: 30.8509 - val_acc: 0.0448
Epoch 193/3000
1338/1338 [==============================] - 0s - loss: 23.6111 - acc: 0.1599 - val_loss: 1.7525 - val_acc: 0.2060
Epoch 194/3000
1338/1338 [==============================] - 0s - loss: 2.8982 - acc: 0.2347 - val_loss: 1.3533 - val_acc: 0.2149
Epoch 195/3000
1338/1338 [==============================] - 0s - loss: 7.8010 - acc: 0.2444 - val_loss: 1.5297 - val_acc: 0.4030
Epoch 196/3000
1338/1338 [==============================] - 0s - loss: 4.9073 - acc: 0.3483 - val_loss: 0.5075 - val_acc: 0.6478
Epoch 197/3000
1338/1338 [==============================] - 0s - loss: 1.7466 - acc: 0.4185 - val_loss: 0.38

1338/1338 [==============================] - 0s - loss: 170.0730 - acc: 0.0456 - val_loss: 26.9830 - val_acc: 0.0000e+00
Epoch 254/3000
1338/1338 [==============================] - 0s - loss: 38.1033 - acc: 0.0792 - val_loss: 11.0300 - val_acc: 0.0149
Epoch 255/3000
1338/1338 [==============================] - 0s - loss: 12.4043 - acc: 0.1981 - val_loss: 49.8351 - val_acc: 0.0030
Epoch 256/3000
1338/1338 [==============================] - 0s - loss: 14.1087 - acc: 0.1054 - val_loss: 0.8998 - val_acc: 0.4806
Epoch 257/3000
1338/1338 [==============================] - 0s - loss: 1.1951 - acc: 0.4477 - val_loss: 2.0856 - val_acc: 0.1075
Epoch 258/3000
1338/1338 [==============================] - 0s - loss: 34.4356 - acc: 0.0762 - val_loss: 89.0570 - val_acc: 0.0000e+00
Epoch 259/3000
1338/1338 [==============================] - 0s - loss: 8.1275 - acc: 0.2638 - val_loss: 1.1672 - val_acc: 0.5463
Epoch 260/3000
1338/1338 [==============================] - 0s - loss: 13.3152 - acc: 0.1854 -

...

1338/1338 [==============================] - 0s - loss: 0.0300 - acc: 0.9828 - val_loss: 0.0164 - val_acc: 0.9881
Epoch 2749/3000
1338/1338 [==============================] - 0s - loss: 0.0203 - acc: 0.9903 - val_loss: 0.0143 - val_acc: 0.9910
Epoch 2750/3000
1338/1338 [==============================] - 0s - loss: 0.0243 - acc: 0.9843 - val_loss: 0.0266 - val_acc: 0.9791
Epoch 2751/3000
1338/1338 [==============================] - 0s - loss: 0.0252 - acc: 0.9821 - val_loss: 0.0103 - val_acc: 0.9910
Epoch 2752/3000
1338/1338 [==============================] - 0s - loss: 0.0216 - acc: 0.9865 - val_loss: 0.0124 - val_acc: 0.9940
Epoch 2753/3000
1338/1338 [==============================] - 0s - loss: 0.0559 - acc: 0.9529 - val_loss: 0.0316 - val_acc: 0.9940
Epoch 2754/3000
1338/1338 [==============================] - 0s - loss: 0.2385 - acc: 0.7586 - val_loss: 0.2108 - val_acc: 0.7522
Epoch 2755/3000
1338/1338 [==============================] - 0s - loss: 0.0798 - acc: 0.9395 - val_loss: 0

1338/1338 [==============================] - 0s - loss: 0.0259 - acc: 0.9836 - val_loss: 0.0154 - val_acc: 0.9940
Epoch 2811/3000
1338/1338 [==============================] - 0s - loss: 0.0249 - acc: 0.9843 - val_loss: 0.0138 - val_acc: 0.9970
Epoch 2812/3000
1338/1338 [==============================] - 0s - loss: 0.0351 - acc: 0.9761 - val_loss: 0.0203 - val_acc: 0.9910
Epoch 2813/3000
1338/1338 [==============================] - 0s - loss: 0.0186 - acc: 0.9880 - val_loss: 0.0150 - val_acc: 0.9910
Epoch 2814/3000
1338/1338 [==============================] - 0s - loss: 0.0164 - acc: 0.9880 - val_loss: 0.0163 - val_acc: 0.9910
Epoch 2815/3000
1338/1338 [==============================] - 0s - loss: 0.0172 - acc: 0.9903 - val_loss: 0.0125 - val_acc: 0.9940
Epoch 2816/3000
1338/1338 [==============================] - 0s - loss: 0.0292 - acc: 0.9776 - val_loss: 0.0355 - val_acc: 0.9851
Epoch 2817/3000
1338/1338 [==============================] - 0s - loss: 0.0391 - acc: 0.9761 - val_loss: 0

1338/1338 [==============================] - 0s - loss: 0.0401 - acc: 0.9746 - val_loss: 0.0151 - val_acc: 0.9881
Epoch 2873/3000
1338/1338 [==============================] - 0s - loss: 0.0172 - acc: 0.9843 - val_loss: 0.0165 - val_acc: 0.9881
Epoch 2874/3000
1338/1338 [==============================] - 0s - loss: 0.0222 - acc: 0.9851 - val_loss: 0.0164 - val_acc: 0.9910
Epoch 2875/3000
1338/1338 [==============================] - 0s - loss: 0.0212 - acc: 0.9843 - val_loss: 0.0108 - val_acc: 0.9881
Epoch 2876/3000
1338/1338 [==============================] - 0s - loss: 0.0258 - acc: 0.9873 - val_loss: 0.0171 - val_acc: 0.9940
Epoch 2877/3000
1338/1338 [==============================] - 0s - loss: 0.0284 - acc: 0.9828 - val_loss: 0.0241 - val_acc: 0.9851
Epoch 2878/3000
1338/1338 [==============================] - 0s - loss: 0.0453 - acc: 0.9694 - val_loss: 0.0384 - val_acc: 0.9851
Epoch 2879/3000
1338/1338 [==============================] - 0s - loss: 0.0186 - acc: 0.9858 - val_loss: 0

1338/1338 [==============================] - 0s - loss: 0.2513 - acc: 0.7683 - val_loss: 0.0274 - val_acc: 0.9731
Epoch 2936/3000
1338/1338 [==============================] - 0s - loss: 0.0400 - acc: 0.9731 - val_loss: 0.0161 - val_acc: 0.9910
Epoch 2937/3000
1338/1338 [==============================] - 0s - loss: 0.2190 - acc: 0.7833 - val_loss: 1.1682 - val_acc: 0.4776
Epoch 2938/3000
1338/1338 [==============================] - 0s - loss: 0.1687 - acc: 0.8640 - val_loss: 0.0786 - val_acc: 0.9493
Epoch 2939/3000
1338/1338 [==============================] - 0s - loss: 0.1979 - acc: 0.8393 - val_loss: 0.0865 - val_acc: 0.9433
Epoch 2940/3000
1338/1338 [==============================] - 0s - loss: 0.0588 - acc: 0.9604 - val_loss: 0.0105 - val_acc: 0.9940
Epoch 2941/3000
1338/1338 [==============================] - 0s - loss: 0.0399 - acc: 0.9806 - val_loss: 0.0926 - val_acc: 0.9493
Epoch 2942/3000
1338/1338 [==============================] - 0s - loss: 0.0847 - acc: 0.9312 - val_loss: 0

1338/1338 [==============================] - 0s - loss: 29.9827 - acc: 0.4305 - val_loss: 240.4082 - val_acc: 0.0000e+00
Epoch 2998/3000
1338/1338 [==============================] - 0s - loss: 304.4288 - acc: 0.0359 - val_loss: 474.6951 - val_acc: 0.0000e+00
Epoch 2999/3000
1338/1338 [==============================] - 0s - loss: 529.4191 - acc: 0.0419 - val_loss: 3.0865 - val_acc: 0.0567
Epoch 3000/3000
1338/1338 [==============================] - 0s - loss: 1.0080 - acc: 0.5665 - val_loss: 0.2259 - val_acc: 0.8448


In [253]:
scores = model.evaluate(X_train.values, Y_train.values) 
scores

1216/1673 [====================>.........] - ETA: 0s

[0.23896379764394993, 0.84040645554047178]

In [282]:
filename_test = "D:/Program/dataset/Taxi Trip Time Prediction/test.csv"

df_t = pd.read_csv(filename_test)
df_t.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"


In [283]:
df_t.shape

(320, 9)

In [284]:
df_t = df_t.drop(["CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TRIP_ID","DAY_TYPE","MISSING_DATA"],axis=1)
df_t['TAXI_ID'] -= np.min(df_t['TAXI_ID']) 
df_t["timecost"] = df_t.apply(caltime,axis=1)
df_t["snapshots"] = df_t.apply(snapshot,axis=1)

# empty_poly = df_t.loc[df_t['POLYLINE'] == '[]'].index.tolist()
# df_t = df_t.drop(empty_poly)

# snap_short = df_t.loc[df_t['snapshots']<4].index.tolist()
# df_t = df_t.drop(snap_short)

df_t['initial_lon'] = df_t.apply(initial_long,axis=1)
df_t['initial_lat'] = df_t.apply(initial_lat,axis=1)
df_t['diff'] = df_t.apply(haversine,axis=1)

dss = df_t.apply(get_date,axis=1)
dss.columns = ["mon","day","hour"]
df_t = df_t.join(dss)

cols = df_t.columns.tolist()
y_label = cols[3]
del cols[3]
cols.append(y_label)
df_t = df_t[cols]

df_t = df_t.drop(["TIMESTAMP"],axis=1)
df_t = df_t.drop(["POLYLINE"],axis=1)
df_t = df_t.drop(["TAXI_ID"],axis=1)

X_test,Y_train = df_t[df.columns[:-1]],df_t[df.columns[-1]]
X_test = X_test.drop(["mon","day"],axis=1)

print(df_t.head())
X_test.head()



   snapshots  initial_lon  initial_lat      diff  mon   day  hour  timecost
0         11    -8.585676    41.148522  0.221328  8.0  14.0  17.0       150
1         40    -8.610876    41.145570  2.140926  8.0  14.0  17.0       585
2         40    -8.585739    41.148558  2.315694  8.0  14.0  17.0       585
3          8    -8.613963    41.141169  0.060305  8.0  14.0  17.0       105
4          2    -8.619903    41.148036  0.000754  8.0  14.0  17.0        15


,snapshots,initial_lon,initial_lat,diff,hour
0,11,-8.585676,41.148522,0.221328,17.0
1,40,-8.610876,41.145570,2.140926,17.0
2,40,-8.585739,41.148558,2.315694,17.0
3,8,-8.613963,41.141169,0.060305,17.0
4,2,-8.619903,41.148036,0.000754,17.0


In [302]:
prediction = model.predict(X_test.values)
prediction = prediction.round()*3


In [303]:
sub = "D:/Program/dataset/Taxi Trip Time Prediction/test.csv"
submission = pd.read_csv(sub)
submission = submission.drop(submission.columns[1:],axis=1)
# sub["TRAVEL_TIME"] = prediction
submission["TRAVEL_TIME"] = prediction
submission.to_csv('submission.csv', index=False)